# LangFuse Data Processing Prompts Optimization

## Mục tiêu học tập
Hiểu cách sử dụng LangFuse để cải thiện độ chính xác và hiệu quả của các prompt dùng cho trích xuất, chuẩn hóa và chuyển đổi dữ liệu bằng LLM.

## Giới thiệu

Việc xử lý dữ liệu phi cấu trúc bằng LLM gặp phải nhiều thách thức:
- **Lỗi định dạng**: LLM có thể trả về dữ liệu không đúng schema mong muốn
- **Thông tin bị bỏ sót**: Một số trường quan trọng có thể bị bỏ qua
- **Hallucination**: LLM có thể tạo ra thông tin không có trong dữ liệu gốc
- **Tính nhất quán**: Kết quả có thể khác nhau giữa các lần chạy

LangFuse giúp giải quyết các vấn đề này bằng cách:
- Theo dõi và phân tích hiệu suất của từng prompt
- Cung cấp dữ liệu để so sánh các phiên bản prompt khác nhau
- Phát hiện các pattern lỗi và cải thiện iterative
- Đánh giá chất lượng output một cách hệ thống

## Cài đặt & Cấu hình

In [ ]:
import os
import json
from datetime import datetime
from typing import Dict, List, Optional, Any
from pydantic import BaseModel, Field, ValidationError
import re

from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser, JsonOutputParser
from langchain_core.messages import HumanMessage
from langfuse import Langfuse
from langfuse.callback import CallbackHandler

# Cấu hình API keys
os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-api-key-here"
os.environ["LANGFUSE_SECRET_KEY"] = "your-langfuse-secret-key"
os.environ["LANGFUSE_PUBLIC_KEY"] = "your-langfuse-public-key"
os.environ["LANGFUSE_HOST"] = "http://localhost:3000"

# Khởi tạo LangFuse
langfuse = Langfuse()
langfuse_handler = CallbackHandler()

print("✅ Cấu hình hoàn tất!")

## Use Case 1: Trích xuất thông tin từ hóa đơn

Chúng ta sẽ xây dựng một hệ thống trích xuất thông tin từ văn bản hóa đơn và theo dõi hiệu suất bằng LangFuse.

In [ ]:
# Định nghĩa schema cho thông tin hóa đơn
class InvoiceItem(BaseModel):
    name: str = Field(description="Tên sản phẩm/dịch vụ")
    quantity: int = Field(description="Số lượng")
    unit_price: float = Field(description="Đơn giá")
    total_price: float = Field(description="Thành tiền")

class InvoiceData(BaseModel):
    invoice_number: str = Field(description="Số hóa đơn")
    date: str = Field(description="Ngày tháng (định dạng YYYY-MM-DD)")
    vendor_name: str = Field(description="Tên nhà cung cấp")
    vendor_address: Optional[str] = Field(description="Địa chỉ nhà cung cấp")
    items: List[InvoiceItem] = Field(description="Danh sách các mục hàng")
    subtotal: float = Field(description="Tổng tiền trước thuế")
    tax_amount: float = Field(description="Số tiền thuế")
    total_amount: float = Field(description="Tổng tiền sau thuế")

# Tạo parser
invoice_parser = PydanticOutputParser(pydantic_object=InvoiceData)

print("✅ Schema định nghĩa hoàn tất!")

In [ ]:
# Dữ liệu hóa đơn mẫu
sample_invoices = [
    """
    HÓA ĐƠN BÁN HÀNG
    Số: HD-2024-001
    Ngày: 15/01/2024
    
    Nhà cung cấp: Công ty TNHH ABC
    Địa chỉ: 123 Đường Nguyễn Văn A, Quận 1, TP.HCM
    
    STT | Tên hàng hóa | Số lượng | Đơn giá | Thành tiền
    1   | Laptop Dell  | 2        | 15000000| 30000000
    2   | Chuột máy tính| 5       | 200000  | 1000000
    3   | Bàn phím     | 3        | 500000  | 1500000
    
    Tổng tiền trước thuế: 32500000
    Thuế VAT (10%): 3250000
    Tổng tiền thanh toán: 35750000
    """,
    
    """
    INVOICE
    Invoice #: INV-2024-0045
    Date: 2024-02-20
    
    Vendor: Tech Solutions Ltd
    Address: 456 Technology St, District 7, Ho Chi Minh City
    
    Description        | Qty | Unit Price | Total
    Software License   | 10  | $299.99    | $2999.90
    Technical Support  | 1   | $500.00    | $500.00
    Training Session   | 2   | $750.00    | $1500.00
    
    Subtotal: $4999.90
    Tax (8%): $399.99
    Total: $5399.89
    """
]

print(f"✅ Chuẩn bị {len(sample_invoices)} hóa đơn mẫu")

In [ ]:
# Tạo prompt template cho trích xuất hóa đơn
invoice_extraction_prompt = PromptTemplate(
    template="""
Bạn là một chuyên gia trích xuất dữ liệu từ hóa đơn. Hãy phân tích văn bản hóa đơn sau và trích xuất thông tin theo định dạng JSON được yêu cầu.

Lưu ý quan trọng:
- Đảm bảo tất cả số tiền được chuyển đổi về đồng VND (nếu có đơn vị khác)
- Ngày tháng phải theo định dạng YYYY-MM-DD
- Nếu thông tin không có, hãy để trống hoặc ghi "N/A"
- Kiểm tra tính toán: subtotal + tax = total

Văn bản hóa đơn:
{invoice_text}

Định dạng output:
{format_instructions}

JSON Output:
""",
    input_variables=["invoice_text"],
    partial_variables={"format_instructions": invoice_parser.get_format_instructions()}
)

print("✅ Prompt template tạo thành công!")

In [ ]:
# Khởi tạo LLM
llm = ChatAnthropic(
    model="claude-3-haiku-20240307",
    temperature=0.1,
    callbacks=[langfuse_handler]
)

# Tạo chain
invoice_chain = invoice_extraction_prompt | llm | invoice_parser

print("✅ Chain tạo thành công!")

In [ ]:
def process_invoice_with_tracing(invoice_text: str, invoice_id: str) -> Dict[str, Any]:
    """
    Xử lý hóa đơn với full tracking qua LangFuse
    """
    # Tạo trace cho toàn bộ quá trình
    trace = langfuse.trace(
        name="invoice_extraction",
        input={"invoice_text": invoice_text[:200] + "..."},
        tags=["data_processing", "invoice", "extraction"],
        metadata={
            "invoice_id": invoice_id,
            "timestamp": datetime.now().isoformat(),
            "text_length": len(invoice_text)
        }
    )
    
    try:
        # Thực hiện trích xuất
        start_time = datetime.now()
        
        result = invoice_chain.invoke({"invoice_text": invoice_text})
        
        end_time = datetime.now()
        processing_time = (end_time - start_time).total_seconds()
        
        # Validation logic
        validation_errors = []
        
        # Kiểm tra tính toán
        calculated_total = result.subtotal + result.tax_amount
        if abs(calculated_total - result.total_amount) > 0.01:
            validation_errors.append(f"Math error: {result.subtotal} + {result.tax_amount} ≠ {result.total_amount}")
        
        # Kiểm tra items
        items_total = sum(item.total_price for item in result.items)
        if abs(items_total - result.subtotal) > 0.01:
            validation_errors.append(f"Items total mismatch: {items_total} ≠ {result.subtotal}")
        
        # Cập nhật trace với kết quả
        trace.update(
            output={
                "extracted_data": result.dict(),
                "validation_errors": validation_errors,
                "processing_time_seconds": processing_time
            }
        )
        
        # Tạo score cho chất lượng trích xuất
        quality_score = 1.0 - (len(validation_errors) * 0.2)
        
        langfuse.score(
            trace_id=trace.id,
            name="extraction_quality",
            value=max(0, quality_score),
            comment=f"Validation errors: {len(validation_errors)}"
        )
        
        return {
            "success": True,
            "data": result,
            "validation_errors": validation_errors,
            "processing_time": processing_time,
            "trace_id": trace.id
        }
        
    except Exception as e:
        # Log lỗi
        trace.update(
            output={"error": str(e)}
        )
        
        langfuse.score(
            trace_id=trace.id,
            name="extraction_quality",
            value=0.0,
            comment=f"Extraction failed: {str(e)}"
        )
        
        return {
            "success": False,
            "error": str(e),
            "trace_id": trace.id
        }

print("✅ Function xử lý hóa đơn với tracing đã sẵn sàng!")

In [ ]:
# Thử nghiệm trích xuất với hóa đơn đầu tiên
print("🔍 Đang xử lý hóa đơn đầu tiên...")

result1 = process_invoice_with_tracing(sample_invoices[0], "invoice_001")

if result1["success"]:
    print("\n✅ Trích xuất thành công!")
    print(f"📊 Trace ID: {result1['trace_id']}")
    print(f"⏱️ Thời gian xử lý: {result1['processing_time']:.2f}s")
    
    # Hiển thị kết quả
    data = result1["data"]
    print(f"\n📋 Thông tin hóa đơn:")
    print(f"- Số HĐ: {data.invoice_number}")
    print(f"- Ngày: {data.date}")
    print(f"- Nhà cung cấp: {data.vendor_name}")
    print(f"- Số mục hàng: {len(data.items)}")
    print(f"- Tổng tiền: {data.total_amount:,.0f} VND")
    
    if result1["validation_errors"]:
        print(f"\n⚠️ Lỗi validation: {result1['validation_errors']}")
    else:
        print("\n✅ Không có lỗi validation")
else:
    print(f"\n❌ Lỗi: {result1['error']}")

In [ ]:
# Xử lý hóa đơn thứ hai
print("🔍 Đang xử lý hóa đơn thứ hai...")

result2 = process_invoice_with_tracing(sample_invoices[1], "invoice_002")

if result2["success"]:
    print("\n✅ Trích xuất thành công!")
    print(f"📊 Trace ID: {result2['trace_id']}")
    print(f"⏱️ Thời gian xử lý: {result2['processing_time']:.2f}s")
    
    # Hiển thị kết quả
    data = result2["data"]
    print(f"\n📋 Thông tin hóa đơn:")
    print(f"- Số HĐ: {data.invoice_number}")
    print(f"- Ngày: {data.date}")
    print(f"- Nhà cung cấp: {data.vendor_name}")
    print(f"- Số mục hàng: {len(data.items)}")
    print(f"- Tổng tiền: {data.total_amount:,.0f} VND")
    
    if result2["validation_errors"]:
        print(f"\n⚠️ Lỗi validation: {result2['validation_errors']}")
    else:
        print("\n✅ Không có lỗi validation")
else:
    print(f"\n❌ Lỗi: {result2['error']}")

## Use Case 2: Chuẩn hóa dữ liệu địa chỉ

Chúng ta sẽ xây dựng hệ thống chuẩn hóa địa chỉ từ nhiều định dạng khác nhau.

In [ ]:
# Schema cho địa chỉ chuẩn hóa
class StandardizedAddress(BaseModel):
    street_number: Optional[str] = Field(description="Số nhà")
    street_name: str = Field(description="Tên đường")
    ward: Optional[str] = Field(description="Phường/Xã")
    district: str = Field(description="Quận/Huyện")
    city: str = Field(description="Thành phố/Tỉnh")
    postal_code: Optional[str] = Field(description="Mã bưu điện")
    country: str = Field(description="Quốc gia", default="Vietnam")
    formatted_address: str = Field(description="Địa chỉ đã được chuẩn hóa")
    confidence_score: float = Field(description="Điểm tin cậy (0-1)", ge=0, le=1)

address_parser = PydanticOutputParser(pydantic_object=StandardizedAddress)

# Dữ liệu địa chỉ mẫu với nhiều định dạng
sample_addresses = [
    "123 Nguyen Van A, Ward 1, District 1, HCMC",
    "456 đường Lê Lợi, phường Bến Nghé, quận 1, thành phố Hồ Chí Minh",
    "789 Le Duan St, Dist 3, Ho Chi Minh City, Vietnam",
    "Số 101, Phố Trần Hưng Đạo, P. Cửa Đông, Q. Hoàn Kiếm, Hà Nội",
    "999 Hai Ba Trung, Da Kao Ward, D1, HCMC 70000"
]

print(f"✅ Chuẩn bị {len(sample_addresses)} địa chỉ mẫu")

In [ ]:
# Prompt cho chuẩn hóa địa chỉ
address_normalization_prompt = PromptTemplate(
    template="""
Bạn là chuyên gia chuẩn hóa địa chỉ Việt Nam. Hãy phân tích địa chỉ sau và chuẩn hóa theo định dạng thống nhất.

Quy tắc chuẩn hóa:
- Chuẩn hóa tên đường (VD: "Nguyen Van A" → "Nguyễn Văn A")
- Chuẩn hóa đơn vị hành chính (VD: "District 1" → "Quận 1", "Ward 1" → "Phường 1")
- Tên thành phố: "Ho Chi Minh City" hoặc "HCMC" → "Thành phố Hồ Chí Minh"
- Tên thủ đô: "Hanoi" → "Hà Nội"
- Đánh giá độ tin cậy dựa trên tính đầy đủ và rõ ràng của thông tin

Địa chỉ cần chuẩn hóa:
{address_text}

Định dạng output:
{format_instructions}

JSON Output:
""",
    input_variables=["address_text"],
    partial_variables={"format_instructions": address_parser.get_format_instructions()}
)

# Tạo chain cho chuẩn hóa địa chỉ
address_chain = address_normalization_prompt | llm | address_parser

print("✅ Chain chuẩn hóa địa chỉ đã sẵn sàng!")

In [ ]:
def normalize_address_with_tracing(address_text: str, address_id: str) -> Dict[str, Any]:
    """
    Chuẩn hóa địa chỉ với full tracking
    """
    trace = langfuse.trace(
        name="address_normalization",
        input={"address_text": address_text},
        tags=["data_processing", "address", "normalization"],
        metadata={
            "address_id": address_id,
            "timestamp": datetime.now().isoformat()
        }
    )
    
    try:
        start_time = datetime.now()
        
        result = address_chain.invoke({"address_text": address_text})
        
        end_time = datetime.now()
        processing_time = (end_time - start_time).total_seconds()
        
        # Validation
        validation_errors = []
        
        # Kiểm tra các trường bắt buộc
        if not result.street_name:
            validation_errors.append("Missing street name")
        if not result.district:
            validation_errors.append("Missing district")
        if not result.city:
            validation_errors.append("Missing city")
        
        # Cập nhật trace
        trace.update(
            output={
                "normalized_address": result.dict(),
                "validation_errors": validation_errors,
                "processing_time_seconds": processing_time
            }
        )
        
        # Score dựa trên confidence và validation
        final_score = result.confidence_score * (1 - len(validation_errors) * 0.1)
        
        langfuse.score(
            trace_id=trace.id,
            name="normalization_quality",
            value=max(0, final_score),
            comment=f"Confidence: {result.confidence_score}, Errors: {len(validation_errors)}"
        )
        
        return {
            "success": True,
            "data": result,
            "validation_errors": validation_errors,
            "processing_time": processing_time,
            "trace_id": trace.id
        }
        
    except Exception as e:
        trace.update(output={"error": str(e)})
        
        langfuse.score(
            trace_id=trace.id,
            name="normalization_quality",
            value=0.0,
            comment=f"Normalization failed: {str(e)}"
        )
        
        return {
            "success": False,
            "error": str(e),
            "trace_id": trace.id
        }

print("✅ Function chuẩn hóa địa chỉ với tracing đã sẵn sàng!")

In [ ]:
# Thực hiện chuẩn hóa cho tất cả địa chỉ mẫu
print("🔍 Đang chuẩn hóa tất cả địa chỉ mẫu...\n")

results = []
for i, address in enumerate(sample_addresses):
    print(f"📍 Địa chỉ {i+1}: {address}")
    
    result = normalize_address_with_tracing(address, f"addr_{i+1:03d}")
    results.append(result)
    
    if result["success"]:
        data = result["data"]
        print(f"   ✅ Chuẩn hóa: {data.formatted_address}")
        print(f"   🎯 Độ tin cậy: {data.confidence_score:.2f}")
        print(f"   ⏱️ Thời gian: {result['processing_time']:.2f}s")
        
        if result["validation_errors"]:
            print(f"   ⚠️ Lỗi: {result['validation_errors']}")
    else:
        print(f"   ❌ Lỗi: {result['error']}")
    
    print()

# Tóm tắt kết quả
successful_results = [r for r in results if r["success"]]
avg_confidence = sum(r["data"].confidence_score for r in successful_results) / len(successful_results) if successful_results else 0
avg_processing_time = sum(r["processing_time"] for r in successful_results) / len(successful_results) if successful_results else 0

print(f"📊 Tóm tắt kết quả:")
print(f"   - Thành công: {len(successful_results)}/{len(results)}")
print(f"   - Độ tin cậy trung bình: {avg_confidence:.2f}")
print(f"   - Thời gian xử lý trung bình: {avg_processing_time:.2f}s")

## Use Case 3: Chuyển đổi định dạng dữ liệu từ văn bản sang JSON

Chuyển đổi thông tin sản phẩm từ mô tả văn bản sang cấu trúc JSON.

In [ ]:
# Schema cho thông tin sản phẩm
class ProductSpecification(BaseModel):
    attribute: str = Field(description="Tên thuộc tính")
    value: str = Field(description="Giá trị thuộc tính")
    unit: Optional[str] = Field(description="Đơn vị đo")

class ProductInfo(BaseModel):
    name: str = Field(description="Tên sản phẩm")
    category: str = Field(description="Danh mục sản phẩm")
    brand: Optional[str] = Field(description="Thương hiệu")
    model: Optional[str] = Field(description="Mã model")
    price: Optional[float] = Field(description="Giá bán")
    currency: str = Field(description="Đơn vị tiền tệ", default="VND")
    specifications: List[ProductSpecification] = Field(description="Thông số kỹ thuật")
    description: str = Field(description="Mô tả sản phẩm")
    availability: str = Field(description="Tình trạng có hàng")

product_parser = PydanticOutputParser(pydantic_object=ProductInfo)

# Dữ liệu sản phẩm mẫu
sample_products = [
    """
    Laptop Gaming ASUS ROG Strix G15 G513QM-HN015W
    
    Thông tin cơ bản:
    - Thương hiệu: ASUS
    - Dòng sản phẩm: ROG Strix G15
    - Model: G513QM-HN015W
    - Giá bán: 25,990,000 VND
    - Tình trạng: Còn hàng
    
    Thông số kỹ thuật:
    - CPU: AMD Ryzen 7 5800H (8 nhân, 16 luồng, 3.2GHz base, 4.4GHz boost)
    - RAM: 16GB DDR4-3200MHz
    - Ổ cứng: 512GB NVMe SSD
    - Card đồ họa: NVIDIA GeForce RTX 3060 6GB GDDR6
    - Màn hình: 15.6 inch Full HD IPS 144Hz
    - Hệ điều hành: Windows 11 Home
    - Trọng lượng: 2.3kg
    - Pin: 56Wh
    
    Mô tả: Laptop gaming mạnh mẽ với thiết kế RGB đẹp mắt, hiệu năng cao cho game và công việc đồ họa.
    """,
    
    """
    iPhone 15 Pro Max 256GB Natural Titanium
    
    Brand: Apple
    Product Line: iPhone 15 Pro Max
    Storage: 256GB
    Color: Natural Titanium
    Price: $1,199 USD
    Status: In Stock
    
    Key Features:
    - Display: 6.7-inch Super Retina XDR OLED
    - Processor: A17 Pro chip with 6-core CPU
    - Camera: 48MP main camera with 5x optical zoom
    - Battery: Up to 29 hours video playback
    - Material: Titanium body with Ceramic Shield front
    - Operating System: iOS 17
    - Connectivity: 5G, Wi-Fi 6E, Bluetooth 5.3
    - Weight: 221g
    
    Description: The most advanced iPhone ever with titanium design and professional camera system.
    """
]

print(f"✅ Chuẩn bị {len(sample_products)} sản phẩm mẫu")

In [ ]:
# Prompt cho chuyển đổi thông tin sản phẩm
product_conversion_prompt = PromptTemplate(
    template="""
Bạn là chuyên gia phân tích thông tin sản phẩm. Hãy trích xuất và cấu trúc hóa thông tin sản phẩm từ văn bản sau.

Lưu ý quan trọng:
- Chuyển đổi giá về VND nếu cần (1 USD = 24,000 VND)
- Phân tích kỹ thông số kỹ thuật và tách thành các thuộc tính riêng biệt
- Xác định đúng danh mục sản phẩm (Laptop, Smartphone, Tablet, v.v.)
- Tình trạng hàng: "Còn hàng", "Hết hàng", "Sắp có hàng"

Thông tin sản phẩm:
{product_text}

Định dạng output:
{format_instructions}

JSON Output:
""",
    input_variables=["product_text"],
    partial_variables={"format_instructions": product_parser.get_format_instructions()}
)

# Tạo chain
product_chain = product_conversion_prompt | llm | product_parser

print("✅ Chain chuyển đổi sản phẩm đã sẵn sàng!")

In [ ]:
def convert_product_with_tracing(product_text: str, product_id: str) -> Dict[str, Any]:
    """
    Chuyển đổi thông tin sản phẩm với full tracking
    """
    trace = langfuse.trace(
        name="product_conversion",
        input={"product_text": product_text[:300] + "..."},
        tags=["data_processing", "product", "conversion"],
        metadata={
            "product_id": product_id,
            "timestamp": datetime.now().isoformat(),
            "text_length": len(product_text)
        }
    )
    
    try:
        start_time = datetime.now()
        
        result = product_chain.invoke({"product_text": product_text})
        
        end_time = datetime.now()
        processing_time = (end_time - start_time).total_seconds()
        
        # Validation
        validation_errors = []
        
        # Kiểm tra các trường bắt buộc
        if not result.name:
            validation_errors.append("Missing product name")
        if not result.category:
            validation_errors.append("Missing category")
        if len(result.specifications) == 0:
            validation_errors.append("No specifications found")
        
        # Kiểm tra logic giá
        if result.price and result.price <= 0:
            validation_errors.append("Invalid price value")
        
        # Cập nhật trace
        trace.update(
            output={
                "converted_product": result.dict(),
                "validation_errors": validation_errors,
                "processing_time_seconds": processing_time,
                "specifications_count": len(result.specifications)
            }
        )
        
        # Score dựa trên completeness
        completeness_score = 0
        if result.name: completeness_score += 0.2
        if result.brand: completeness_score += 0.1
        if result.price: completeness_score += 0.2
        if result.specifications: completeness_score += 0.3
        if result.description: completeness_score += 0.2
        
        final_score = completeness_score * (1 - len(validation_errors) * 0.1)
        
        langfuse.score(
            trace_id=trace.id,
            name="conversion_quality",
            value=max(0, final_score),
            comment=f"Completeness: {completeness_score:.2f}, Errors: {len(validation_errors)}"
        )
        
        return {
            "success": True,
            "data": result,
            "validation_errors": validation_errors,
            "processing_time": processing_time,
            "completeness_score": completeness_score,
            "trace_id": trace.id
        }
        
    except Exception as e:
        trace.update(output={"error": str(e)})
        
        langfuse.score(
            trace_id=trace.id,
            name="conversion_quality",
            value=0.0,
            comment=f"Conversion failed: {str(e)}"
        )
        
        return {
            "success": False,
            "error": str(e),
            "trace_id": trace.id
        }

print("✅ Function chuyển đổi sản phẩm với tracing đã sẵn sàng!")

In [ ]:
# Thực hiện chuyển đổi cho tất cả sản phẩm mẫu
print("🔍 Đang chuyển đổi thông tin sản phẩm...\n")

product_results = []
for i, product in enumerate(sample_products):
    print(f"📱 Sản phẩm {i+1}:")
    
    result = convert_product_with_tracing(product, f"prod_{i+1:03d}")
    product_results.append(result)
    
    if result["success"]:
        data = result["data"]
        print(f"   ✅ Tên: {data.name}")
        print(f"   🏷️ Danh mục: {data.category}")
        print(f"   🏢 Thương hiệu: {data.brand or 'N/A'}")
        print(f"   💰 Giá: {data.price:,.0f} {data.currency}" if data.price else "   💰 Giá: N/A")
        print(f"   📋 Thông số: {len(data.specifications)} mục")
        print(f"   📊 Độ hoàn thiện: {result['completeness_score']:.2f}")
        print(f"   ⏱️ Thời gian: {result['processing_time']:.2f}s")
        
        if result["validation_errors"]:
            print(f"   ⚠️ Lỗi: {result['validation_errors']}")
    else:
        print(f"   ❌ Lỗi: {result['error']}")
    
    print()

# Tóm tắt kết quả
successful_products = [r for r in product_results if r["success"]]
avg_completeness = sum(r["completeness_score"] for r in successful_products) / len(successful_products) if successful_products else 0
avg_specs_count = sum(len(r["data"].specifications) for r in successful_products) / len(successful_products) if successful_products else 0

print(f"📊 Tóm tắt kết quả chuyển đổi sản phẩm:")
print(f"   - Thành công: {len(successful_products)}/{len(product_results)}")
print(f"   - Độ hoàn thiện trung bình: {avg_completeness:.2f}")
print(f"   - Số thông số trung bình: {avg_specs_count:.1f}")

## Phân tích & Cải thiện Prompt với LangFuse

Bây giờ chúng ta sẽ sử dụng LangFuse UI để phân tích hiệu suất và cải thiện prompt.

In [ ]:
# Tạo session để nhóm tất cả các traces
session = langfuse.create_session(
    name="data_processing_optimization",
    metadata={
        "experiment_type": "prompt_optimization",
        "timestamp": datetime.now().isoformat(),
        "total_operations": len(sample_invoices) + len(sample_addresses) + len(sample_products)
    }
)

print(f"📊 Session tạo thành công: {session.id}")
print(f"🔗 Xem chi tiết tại: http://localhost:3000/sessions/{session.id}")

# Flush để đảm bảo tất cả dữ liệu được gửi
langfuse.flush()

print("\n✅ Tất cả dữ liệu đã được gửi lên LangFuse!")

In [ ]:
# Lấy thống kê từ LangFuse
def get_performance_stats():
    """
    Lấy thống kê hiệu suất từ LangFuse
    """
    try:
        # Lấy tất cả traces từ session
        traces = langfuse.get_traces(limit=100)
        
        # Phân loại theo tags
        invoice_traces = []
        address_traces = []
        product_traces = []
        
        for trace in traces.data:
            if 'invoice' in trace.tags:
                invoice_traces.append(trace)
            elif 'address' in trace.tags:
                address_traces.append(trace)
            elif 'product' in trace.tags:
                product_traces.append(trace)
        
        stats = {
            "invoice_extraction": {
                "count": len(invoice_traces),
                "avg_latency": sum(t.latency for t in invoice_traces if t.latency) / len(invoice_traces) if invoice_traces else 0
            },
            "address_normalization": {
                "count": len(address_traces),
                "avg_latency": sum(t.latency for t in address_traces if t.latency) / len(address_traces) if address_traces else 0
            },
            "product_conversion": {
                "count": len(product_traces),
                "avg_latency": sum(t.latency for t in product_traces if t.latency) / len(product_traces) if product_traces else 0
            }
        }
        
        return stats
    
    except Exception as e:
        print(f"Lỗi khi lấy thống kê: {e}")
        return None

# Hiển thị thống kê
stats = get_performance_stats()
if stats:
    print("📈 Thống kê hiệu suất:")
    print(f"\n🧾 Trích xuất hóa đơn:")
    print(f"   - Số traces: {stats['invoice_extraction']['count']}")
    print(f"   - Latency trung bình: {stats['invoice_extraction']['avg_latency']:.2f}ms")
    
    print(f"\n📍 Chuẩn hóa địa chỉ:")
    print(f"   - Số traces: {stats['address_normalization']['count']}")
    print(f"   - Latency trung bình: {stats['address_normalization']['avg_latency']:.2f}ms")
    
    print(f"\n📱 Chuyển đổi sản phẩm:")
    print(f"   - Số traces: {stats['product_conversion']['count']}")
    print(f"   - Latency trung bình: {stats['product_conversion']['avg_latency']:.2f}ms")
else:
    print("⚠️ Không thể lấy thống kê từ LangFuse")

## Hướng dẫn sử dụng LangFuse UI để cải thiện prompt

### 1. Truy cập LangFuse Dashboard
- Mở trình duyệt và truy cập: `http://localhost:3000`
- Đăng nhập với tài khoản của bạn

### 2. Phân tích Traces
- Trong tab **Traces**, tìm các traces với tags `data_processing`
- Sắp xếp theo **Score** để xem các kết quả tốt nhất và tệ nhất
- Click vào từng trace để xem chi tiết:
  - Input: Dữ liệu đầu vào
  - Output: Kết quả trích xuất
  - Metadata: Thông tin bổ sung (thời gian, lỗi validation)

### 3. Xác định Pattern lỗi
- **Lỗi thường gặp với hóa đơn:**
  - Sai định dạng ngày tháng
  - Nhầm lẫn đơn vị tiền tệ
  - Thiếu thông tin thuế
- **Lỗi thường gặp với địa chỉ:**
  - Không chuẩn hóa được tên địa danh
  - Thiếu thông tin quận/huyện
  - Confidence score thấp
- **Lỗi thường gặp với sản phẩm:**
  - Bỏ sót thông số kỹ thuật
  - Không chuyển đổi được đơn vị
  - Phân loại sai danh mục

### 4. Cải thiện Prompt
Dựa trên phân tích, bạn có thể:
- **Thêm ví dụ cụ thể** trong prompt cho các trường hợp khó
- **Cải thiện instruction** để xử lý edge cases
- **Thêm validation rules** trong prompt
- **Sử dụng few-shot learning** với các ví dụ thành công

### 5. A/B Testing
- Tạo phiên bản prompt mới
- Chạy thử nghiệm với cùng dữ liệu
- So sánh scores giữa các phiên bản
- Chọn phiên bản có hiệu suất tốt nhất

In [ ]:
# Ví dụ về cải thiện prompt dựa trên feedback
improved_invoice_prompt = PromptTemplate(
    template="""
Bạn là một chuyên gia trích xuất dữ liệu từ hóa đơn với 10 năm kinh nghiệm. Hãy phân tích văn bản hóa đơn sau và trích xuất thông tin theo định dạng JSON được yêu cầu.

QUAN TRỌNG - Quy tắc xử lý:
1. NGÀY THÁNG: 
   - Chuyển đổi tất cả về định dạng YYYY-MM-DD
   - VD: "15/01/2024" → "2024-01-15"
   - VD: "Jan 15, 2024" → "2024-01-15"

2. TIỀN TỆ:
   - Tất cả số tiền phải chuyển về VND
   - 1 USD = 24,000 VND
   - 1 EUR = 26,000 VND
   - Loại bỏ dấu phẩy và ký hiệu tiền tệ

3. VALIDATION:
   - Kiểm tra: Tổng các items = Subtotal
   - Kiểm tra: Subtotal + Tax = Total
   - Nếu không khớp, ưu tiên số liệu từ bảng chi tiết

4. THÔNG TIN THIẾU:
   - Nếu không tìm thấy thông tin, ghi "N/A"
   - Không được tự tạo ra thông tin không có

Văn bản hóa đơn:
{invoice_text}

Định dạng output:
{format_instructions}

Hãy kiểm tra kỹ tất cả phép tính trước khi trả về kết quả.

JSON Output:
""",
    input_variables=["invoice_text"],
    partial_variables={"format_instructions": invoice_parser.get_format_instructions()}
)

print("✅ Prompt cải thiện đã sẵn sàng!")
print("\n🔄 Bạn có thể thử nghiệm prompt mới này với dữ liệu tương tự và so sánh kết quả trong LangFuse.")

## Tài liệu Tham khảo

### LangFuse Documentation
- **Prompt Management**: https://langfuse.com/docs/prompts
- **Evaluation & Scoring**: https://langfuse.com/docs/scores
- **Tracing & Observability**: https://langfuse.com/docs/tracing
- **LangChain Integration**: https://langfuse.com/docs/integrations/langchain
- **Dashboard Analytics**: https://langfuse.com/docs/analytics

### Best Practices
- **Prompt Engineering**: https://langfuse.com/docs/prompts/best-practices
- **Data Quality Monitoring**: https://langfuse.com/docs/data-quality
- **Performance Optimization**: https://langfuse.com/docs/performance

### API References
- **Python SDK**: https://langfuse.com/docs/sdk/python
- **REST API**: https://langfuse.com/docs/api

## Kết luận & Bước tiếp theo

### Những gì đã học được:
1. **Cách sử dụng LangFuse để theo dõi hiệu suất** của các prompt xử lý dữ liệu
2. **Thiết kế validation logic** để đảm bảo chất lượng dữ liệu đầu ra
3. **Phân tích pattern lỗi** và cải thiện prompt dựa trên dữ liệu thực tế
4. **Sử dụng scoring system** để đánh giá chất lượng tự động
5. **Tạo metadata phong phú** để phân tích sâu hơn

### Bước tiếp theo:
1. **Mở rộng validation rules** cho các use case cụ thể
2. **Thực hiện A/B testing** với các phiên bản prompt khác nhau
3. **Tích hợp human feedback** để cải thiện liên tục
4. **Xây dựng pipeline tự động** cho việc retraining prompt
5. **Triển khai monitoring** trong production environment

### Key Takeaways:
- **LangFuse là công cụ mạnh mẽ** để tối ưu hóa prompt cho data processing
- **Validation và scoring** là chìa khóa để đảm bảo chất lượng
- **Iterative improvement** dựa trên dữ liệu thực tế mang lại kết quả tốt nhất
- **Structured output parsing** giúp đảm bảo tính nhất quán của dữ liệu

### Thực hành tiếp theo:
Hãy thử áp dụng những kỹ thuật này vào dự án thực tế của bạn và sử dụng LangFuse để liên tục cải thiện hiệu suất!